In [28]:
"""
Analyze JSON logs of PPA runs to diagnose what is failing.
# NOTE - to run this script, must be logged in to admin or other account authorized to read from the VGIS workflow folder
    on the PPA server machine
"""

import datetime as dt
from pathlib import Path
import json
from copy import deepcopy

import pandas as pd

time_zone = 'US/Pacific' # pytz timezone .tz_convert('US/Pacific')

def parse_json(json_in, run_id=0):
    with open(json_in, 'r') as f:
        jloaded = json.load(f)
        
    out_rows = []
    
    base_row = {}
    base_row['run_id'] = run_id
    base_row['log_path'] = str(json_in)
    base_row['start_time'] = pd.to_datetime(jloaded['start']).tz_convert(time_zone)
    base_row['projectType'] = 'unknown'
    base_row['projectName'] = 'unknown'
    base_row['rpt_name'] = 'unknown'
    base_row['email'] = 'unknown'
    base_row['error_flag'] = 1
    base_row['error_msg'] = 'Other error. See JSON for details.'
    
    try:
        jbase = jloaded['artifacts'][0]['outputs']['output']
        base_row['projectType'] = jbase['projectType']
        base_row['projectName'] = jbase['projectName']
        base_row['email'] = jbase['email']
        
        for i, rpt in enumerate(jbase['reports']):
            rpt_row = deepcopy(base_row) # must do deepcopy so that base_row itself does not change
            rpt_row['rpt_name'] = rpt['name']

            rpt_error = False
            rpt_errmsg = ''
            messages = rpt['gpJob']['messages']
            if len(messages) > 0:
                for msg in messages:
                    if msg['type'] == 'esriJobMessageTypeError':
                        rpt_error = True
                    if rpt_error and msg['type'] == 'esriJobMessageTypeInformative':
                        rpt_errmsg = msg['description']
                        
            if rpt.get('error') is not None: # sometimes the error is not stored in the 'messages'
                rpt_error = True
                errmsg2 = rpt['error']['error']
                errmsgs = [rpt_errmsg, errmsg2] if rpt_errmsg != '' else [errmsg2]
                rpt_errmsg = '; '.join(errmsgs)

            rpt_row['error_flag'] = int(rpt_error)
            rpt_row['error_msg'] = rpt_errmsg

            out_rows.append(rpt_row)
            
    except KeyError: # if key error, means significant error and JSON log did not generate as normal
        out_rows.append(base_row)

        
    # if 'df74795d5bf043d89e190229afca4446' in str(json_in): import pdb; pdb.set_trace()
    return out_rows

print("modules loaded.")

# test_json

# tjson = r'\\WEBMAPPING-SVR\Workflow\5\5950b5cf93354d7e9482d469a15ba6e3\Logs\df74795d5bf043d89e190229afca4446.json'
# parse_json(tjson)
# df.loc[df['log_path'].str.contains('df74795d5bf043d89e190229afca4446')]

modules loaded.


In [29]:
# NOTE - to run this script, must be logged in to admin or other account authorized to read from the VGIS workflow folder
root_dir = Path(r'\\WEBMAPPING-SVR\Workflow')
subdirs = ['0', '1', '2', '3', '4', '5', '6', '7', '8'] # subfolder names within Workflow folder

null_row = {'run_id': 0,
            'log_path': 'na',
            'start_time': 'na',
            'projectType': 'unknown',
            'projectName': 'unknown',
            'rpt_name': 'unknown',
            'email': 'unknown',
            'error_flag': 1,
            'error_msg': 'Other error. See JSON for details.'
           }

data_rows = []
run_n = 0
for sd in subdirs:
    dpth = root_dir.joinpath(sd)
    logpaths = [i.joinpath('Logs') for i in dpth.glob('*') if i.is_dir()]
    for lp in logpaths:
        log_files = [i for i in lp.glob('*.json')]
        if len(log_files) > 1:
            null_row['log_path'] = str(lp)
            null_row['run_id'] = run_n
            
            modtime = dt.datetime.fromtimestamp(lp.lstat().st_mtime)
            null_row['start_time'] = pd.to_datetime(modtime).tz_localize(time_zone)
            data_rows.append(null_row)
        else:
            log_data = parse_json(log_files[0], run_id=run_n)
            data_rows += log_data
            run_n += 1


df = pd.DataFrame(data_rows)
unique_logs = df.run_id.drop_duplicates().shape[0]
print(f"{unique_logs} run logs summarized")




1008 run logs summarized


In [32]:
# Look at the last 10 errors
df.loc[df['error_flag'] == 1].sort_values(by='start_time', ascending=False).head(10)

,run_id,log_path,start_time,projectType,projectName,rpt_name,email,error_flag,error_msg
2417,496,\\WEBMAPPING-SVR\Workflow\4\68078350e9c84521a4...,2024-09-27 11:39:51.913977600-07:00,Freeway Expansion,test,Promote Economic Prosperity,dconly@sacog.org,1,No report data collected from gp task
577,112,\\WEBMAPPING-SVR\Workflow\0\d886bf069ec04bbaa6...,2024-09-23 16:12:28.121727500-07:00,Arterial or Transit Expansion,test,Promote Socioeconomic Equity,dconly@sacog.org,1,esriJobFailed
576,112,\\WEBMAPPING-SVR\Workflow\0\d886bf069ec04bbaa6...,2024-09-23 16:12:28.121727500-07:00,Arterial or Transit Expansion,test,Promote Complete Streets and State of Good Repair,dconly@sacog.org,1,esriJobFailed
2913,603,\\WEBMAPPING-SVR\Workflow\5\5950b5cf93354d7e94...,2024-09-19 06:42:56.180301900-07:00,Arterial State of Good Repair,Lincoln Blvd,Promote Economic Prosperity,cgillis@ben-en.com,1,No report data collected from gp task
1873,379,\\WEBMAPPING-SVR\Workflow\3\91afa86c33d64798bd...,2024-09-17 08:52:57.701764700-07:00,Arterial State of Good Repair,Broadway Complete Streets,Promote Economic Prosperity,cdougherty@sacog.org,1,java.lang.RuntimeException: com.esri.arcgis.di...
1875,379,\\WEBMAPPING-SVR\Workflow\3\91afa86c33d64798bd...,2024-09-17 08:52:57.701764700-07:00,Arterial State of Good Repair,Broadway Complete Streets,Make a Safer Transportation System,cdougherty@sacog.org,1,No report data collected from gp task
1876,379,\\WEBMAPPING-SVR\Workflow\3\91afa86c33d64798bd...,2024-09-17 08:52:57.701764700-07:00,Arterial State of Good Repair,Broadway Complete Streets,Promote Complete Streets and State of Good Repair,cdougherty@sacog.org,1,java.lang.RuntimeException: com.esri.arcgis.di...
1877,379,\\WEBMAPPING-SVR\Workflow\3\91afa86c33d64798bd...,2024-09-17 08:52:57.701764700-07:00,Arterial State of Good Repair,Broadway Complete Streets,Promote Socioeconomic Equity,cdougherty@sacog.org,1,No report data collected from gp task
1948,392,\\WEBMAPPING-SVR\Workflow\3\a7896915b558453ba2...,2024-09-17 08:51:56.432022100-07:00,Arterial State of Good Repair,test,Reduce Congestion,dconly@sacog.org,1,java.lang.RuntimeException: com.esri.arcgis.di...
1947,392,\\WEBMAPPING-SVR\Workflow\3\a7896915b558453ba2...,2024-09-17 08:51:56.432022100-07:00,Arterial State of Good Repair,test,Reduce VMT,dconly@sacog.org,1,java.lang.RuntimeException: com.esri.arcgis.di...


In [31]:
# compare before and after 6/12 migration to new PPA server

before_migrn = df.loc[df['start_time'] <= '2024-06-12'].sort_values(by='start_time')
after_migrn = df.loc[df['start_time'] > '2024-06-12'].sort_values(by='start_time')

display(pd.DataFrame(before_migrn.error_msg.value_counts()))
print("\nafter migration to new server")
display(pd.DataFrame(after_migrn.error_msg.value_counts()))

,count
error_msg,
,3791
esriJobFailed,155
Other error. See JSON for details.,120
Gp task has no job id.,30
java.lang.Exception: Could not service request.; esriJobFailed,8
No report data collected from gp task,5
java.lang.Exception: Processing request took longer than the usage timeout for service 'RPArtExpEquity.GPServer'.; esriJobFailed,1
java.lang.Exception: java.lang.RuntimeException: Failed to load C:\Program Files\ArcGIS\Server\framework\runtime\ArcGIS\\bin\DADFLib.dll. Error code 1114; esriJobFailed,1



after migration to new server


,count
error_msg,
,676
Other error. See JSON for details.,31
esriJobFailed,12
No report data collected from gp task,8
java.lang.RuntimeException: com.esri.arcgis.discovery.admin.rediscovery.WaitTimeoutException: Wait timeout for the request exceeded.; esriJobFailed,4
java.lang.RuntimeException: java.util.concurrent.TimeoutException; esriJobFailed,3


In [39]:
# after 6/12 migration, how many errors per GP task? Are any specific GP tasks more prone to errors?
after_migrn = df.loc[df['start_time'] > '2024-06-12'].sort_values(by='start_time')

after_migrn.groupby(['projectType', 'rpt_name'])['error_flag'].sum().reset_index()
dfgb = after_migrn.groupby(['projectType', 'rpt_name']) \
.agg(errcnt=('error_flag', 'sum'), runcnt=('error_flag', 'count')).reset_index()

dfgb['err_rate'] = dfgb['errcnt'] / dfgb['runcnt']
dfgb.sort_values(by='err_rate', ascending=False)

,projectType,rpt_name,errcnt,runcnt,err_rate
30,unknown,unknown,2,2,1.000000
8,Arterial State of Good Repair,unknown,21,21,1.000000
17,Arterial or Transit Expansion,unknown,8,8,1.000000
2,Arterial State of Good Repair,Make a Safer Transportation System,10,61,0.163934
27,Freeway Expansion,Promote Economic Prosperity,1,7,0.142857
14,Arterial or Transit Expansion,Promote Socioeconomic Equity,2,17,0.117647
12,Arterial or Transit Expansion,Promote Complete Streets and State of Good Repair,1,12,0.083333
4,Arterial State of Good Repair,Promote Economic Prosperity,4,69,0.057971
9,Arterial or Transit Expansion,Encourage Multimodal Travel,1,20,0.050000
5,Arterial State of Good Repair,Promote Socioeconomic Equity,2,53,0.037736
